In [1]:
# imports
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer


# allow plots to appear directly in the notebook
import seaborn as sns
%matplotlib inline

In [16]:
data = pd.read_csv(r'C:\Users\alsherman\Desktop\GitHub\DataScience_GeneralAssembly\Data\Cleaned_Data_Final.csv')

In [17]:
data.describe()

,ID,country,latitude,location_data_accuracy,longitude,availability,average_image_size,bathroom,bedroom,cat,...,price,smoking,square_footage,grocery_list,gym_list,square_footage_cleaned,weekday_of_posting,Posting_Time_AM_or_PM,Posting_Time,Posting_Day
count,5.630000e+03,5630.000000,5630.000000,5630.000000,5630.000000,5630,5630.000000,5630.000000,5630.000000,5630.000000,...,5630.000000,5630.000000,5630.000000,5630.000000,5630.000000,5630.00000,5630.000000,5630.000000,5630.000000,5630.000000
mean,5.000924e+09,0.625755,30.527165,3.727531,-60.463262,1,233024.866785,1.459059,1.730551,0.600533,...,1904.604618,0.695737,808.919716,10.374778,7.287567,993.95222,4.322380,0.666607,6.768060,8.976199
std,1.053905e+07,0.483970,16.076215,6.949249,31.857710,0,92831.331774,0.792666,1.122790,0.489832,...,824.794164,0.460135,704.601661,8.168322,7.534812,588.27218,1.664735,0.471467,3.729488,10.291046
min,4.959352e+09,0.000000,-85.050342,0.000000,-96.045800,1,0.000000,0.000000,0.000000,0.000000,...,129.000000,0.000000,0.000000,0.000000,0.000000,58.00000,0.000000,0.000000,1.000000,1.000000
25%,4.999076e+09,0.000000,38.770700,0.000000,-77.111260,1,270000.000000,1.000000,1.000000,0.000000,...,1395.000000,0.000000,460.500000,1.000000,0.000000,680.00000,4.000000,0.000000,3.280000,2.000000
50%,5.006130e+09,1.000000,38.895742,0.000000,-77.038839,1,270000.000000,1.000000,2.000000,1.000000,...,1728.000000,1.000000,786.000000,11.000000,4.000000,874.00000,5.000000,1.000000,6.530000,3.000000
75%,5.006978e+09,1.000000,38.961500,5.000000,-76.869100,1,270000.000000,2.000000,2.000000,1.000000,...,2200.000000,1.000000,1080.000000,20.000000,14.000000,1146.75000,6.000000,1.000000,10.260000,18.000000
max,5.008243e+09,1.000000,41.333300,22.000000,179.992769,1,270000.000000,5.500000,6.000000,1.000000,...,8142.000000,1.000000,18711.000000,20.000000,20.000000,18711.00000,6.000000,1.000000,12.590000,30.000000


In [18]:
# Create dummy variables

data = pd.concat([data, pd.get_dummies(data.housing_type)], axis=1)
data = pd.concat([data, pd.get_dummies(data.laundry)], axis=1)
data = pd.concat([data, pd.get_dummies(data.parking)], axis=1)

data[0:1]

,ID,city,country,latitude,location_data_accuracy,longitude,state,availability,average_image_size,bathroom,...,laundry on site,w/d hookups,w/d in unit,No_Parking_Data,attached garage,carport,detached garage,off-street parking,street parking,valet parking
0,4959351766,No_City_Data,0,0,0,0,No_State_Data,1,270000,1,...,0,0,0,1,0,0,0,0,0,0


In [19]:
data.columns

Index([u'ID', u'city', u'country', u'latitude', u'location_data_accuracy', u'longitude', u'state', u'availability', u'average_image_size', u'bathroom', u'bedroom', u'cat', u'date_available', u'description', u'dog', u'housing_type', u'image_number', u'laundry', u'parking', u'price', u'smoking', u'square_footage', u'url', u'grocery_list', u'gym_list', u'square_footage_cleaned', u'time_of_day_of_posting', u'date_of_posting', u'weekday_of_posting', u'Posting_Time_AM_or_PM', u'Posting_Time', u'Posting_Day', u'No_Housing_Data', u'apartment', u'condo', u'cottage/cabin', u'duplex', u'flat', u'house', u'in-law', u'townhouse', u'No_Laundry_Data', u'laundry in bldg', u'laundry on site', u'w/d hookups', u'w/d in unit', u'No_Parking_Data', u'attached garage', u'carport', u'detached garage', u'off-street parking', u'street parking', u'valet parking'], dtype='object')

In [23]:
feature_cols = [
'latitude',
'longitude',
'bathroom',
'bedroom',
'image_number',
#'square_footage', 
'square_footage_cleaned',
'grocery_list',
'gym_list',
#'dc',    
'attached garage',
'off-street parking',
'street parking',     
'Posting_Time_AM_or_PM',
'Posting_Time',
'Posting_Day',
'weekday_of_posting'
]

## Train Test Split

In [24]:
# create X and y
X = data[feature_cols]
y = data.price

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

## Linear Regression

In [25]:
### SCIKIT-LEARN ###

# instantiate and fit
lm2 = LinearRegression()
lm2.fit(X_train, y_train)

y_pred = lm2.predict(X_test) # Predict
# print the intercept
#print lm2.intercept_

# pair the feature names with the coefficients
#print zip(feature_cols, lm2.coef_)

# Access accuracy
y_pred = [int(y) for y in y_pred]
y_test = [int(y) for y in y_test]

print metrics.accuracy_score(y_test, y_pred)

0.00284090909091


##Random Forest

In [27]:
# import class, instantiate estimator, fit with all data
rfclf = RandomForestClassifier(n_estimators=200, max_features=5, oob_score=True, random_state=1)
rfclf.fit(X_train, y_train)

# compute the out-of-bag classification accuracy
print rfclf.oob_score_
print len(feature_cols)
print len(rfclf.feature_importances_)

0.178588346755
15
15


In [28]:
# compute the feature importances
Feature_Importance = pd.DataFrame({'feature':feature_cols, 'importance':rfclf.feature_importances_})
Feature_Importance.sort('importance', ascending=False)

,feature,importance
5,square_footage_cleaned,0.149494
12,Posting_Time,0.145125
4,image_number,0.105619
0,latitude,0.104761
1,longitude,0.103833
13,Posting_Day,0.073015
7,gym_list,0.058480
14,weekday_of_posting,0.058181
6,grocery_list,0.056977
3,bedroom,0.036880


##Naive Bayes

In [195]:
#Create Train Test split for text data (listing description)

naive_bayes_feature_cols = 'description'

# create X and y
X = data[naive_bayes_feature_cols]
y = data.price

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [196]:
# COUNTVECTORIZER: 'convert text into a matrix of token counts'
# instantiate the vectorizer
vect = CountVectorizer()

# learn vocabulary and create document-term matrix in a single step
train_dtm = vect.fit_transform(X_train)
train_dtm

# transform testing data into a document-term matrix
test_dtm = vect.transform(X_test)
test_dtm

# store feature names and examine them
train_features = vect.get_feature_names()
len(train_features)
train_features[:50]
train_features[-50:]

# convert train_dtm to a regular array
train_arr = train_dtm.toarray()
train_arr

# examine the vocabulary and document-term matrix together
pd.DataFrame(train_dtm.toarray(), columns=vect.get_feature_names()).head()

,00,000,0009,000pets,001114,001255887315,001368910757,002,004,005,...,zumba,zy,zyq9z,zz,¼gather,área,â¼,â½,über,ýntelligence
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [197]:
nb = MultinomialNB()
nb.fit(train_dtm, y_train)

# make predictions on test data using test_dtm
y_pred = nb.predict(test_dtm)
y_pred

# compare predictions to true labels
print metrics.accuracy_score(y_test, y_pred)
#print metrics.confusion_matrix(y_test, y_pred)

0.227984873042


## Logistic Regression

In [117]:
# Fit model
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test) # Predict

# Access accuracy
print metrics.accuracy_score(y_test, y_pred)

0.116153430578


In [89]:
# Create a Confusion Matrix

con_mat = metrics.confusion_matrix(y_test, y_pred)
print con_mat

[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
